In [1]:
import sys
import numpy as np
import pandas as pd
import os
from torch.utils.data import DataLoader, WeightedRandomSampler
from torch.utils.data.dataloader import default_collate
import torch.nn as nn
import torch.optim as optim
from torchsummary import summary
import torch
import pickle

# Append the project dir to path
sys.path.append(os.path.join("..", "..", ".."))
from data_pipeline.utils import train_test_split, create_metadata_df, get_sample_weights_of_dataset, measurements_to_df
from data_pipeline.dataset_xy import CARLADatasetXY
from data_pipeline.dataset_xy_opt import CARLADatasetXYOpt
from data_pipeline.data_sampler import BranchPerCommandSampler
from data_pipeline.data_preprocessing import preprocessing
from models.resnet_baseline.architectures_v3 import Resnet_Baseline_V3, Resnet_Baseline_V3_Dropout
from models.model_trainer import ModelTrainer

/Users/julianvonklitzing/miniforge3/envs/carla/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Set data balancing options (if both false, then no balancing is applied)
use_balance_by_loss_weighting = False
use_balance_by_over_under_sampling = True

assert not use_balance_by_loss_weighting or not use_balance_by_over_under_sampling

In [3]:
# Train additionally on the noisy data
use_data_noisy = True

path_data_noisy = None
if use_data_noisy:
    path_data_noisy = os.path.join("..", "..", "..", "data", "Noise-Dataset")

In [5]:
path_data = os.path.join("..", "..", "..", "data", "data")

config_xy = {"used_inputs": ["rgb", "measurements"], 
        "used_measurements": ["speed", "steer", "throttle", "brake", "command"],
        "y": ["brake", "steer", "throttle"],
        "seq_len": 1
        }

In [7]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'mps' if torch.has_mps else 'cpu')
batch_size = 64

# Create df_meta 
df_meta_data = create_metadata_df(path_data, config_xy["used_inputs"])
df_meta_data_noisy = None
if use_data_noisy:
    df_meta_data_noisy = create_metadata_df(path_data_noisy, config_xy["used_inputs"])

# Train/test split
train_test_config = {
    "train": ['Town00', 'Town01', 'Town02', 'Town03', 'Town04', 'Town05', 'Town07', 'Town08', 'Town09', 'Town10'],
    "test": ['Town06']
}
df_meta_data_train, df_meta_data_test_1, df_meta_data_test_2 = train_test_split(df_meta_data, towns_intersect=train_test_config, df_meta_data_noisy=df_meta_data_noisy)

# Decrease train/test size for quick test run
df_meta_data_train = df_meta_data_train.head(5 * batch_size)
df_meta_data_test_1 = df_meta_data_test_1.head(5 * batch_size)
df_meta_data_test_2 = df_meta_data_test_2.head(5 * batch_size)

# Create Dataset & DataLoader
dataset_train = CARLADatasetXY(root_dir=path_data, df_meta_data=df_meta_data_train, config=config_xy)
dataset_test_1 = CARLADatasetXY(root_dir=path_data, df_meta_data=df_meta_data_test_1, config=config_xy)
dataset_test_2 = CARLADatasetXY(root_dir=path_data, df_meta_data=df_meta_data_test_2, config=config_xy)

dataset = CARLADatasetXYOpt(df_meta_data)

Varying number files among input types: ../../../data/data/rl_dataset_23_11/Routes_clipped_Town05_rl_Seed0/clipped_Town05_rl_route30_11_23_17_42_15/lidar_trans


In [11]:
x, y, idx = dataset.__getitem__(0)

In [14]:
way_np = y["waypoints"]
way_torch = torch.Tensor(way_np)

In [16]:
def prep_waypoints(waypoints, x, y, theta):
    waypoints = np.asarray(waypoints)
    waypoints = waypoints[:, :2]
    R = np.array([
            [np.cos(theta), -np.sin(theta)],
            [np.sin(theta), np.cos(theta)]
            ])

    local_command_points = np.array([waypoints[:,0]-x, waypoints[:,1]-y])
    local_command_points = R.T.dot(local_command_points)
    return torch.from_numpy(local_command_points.T)